In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.models as models
import cv2
import sys
import math
import random
import splitfolders
import torchsummary
from tqdm import tqdm
from ResNet_18 import resnet
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [3]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':50,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':32,
    'SEED':42
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

In [5]:
train_df = pd.read_csv('./train_data.csv', index_col = 0)
test_df = pd.read_csv('./test_data.csv', index_col = 0)

In [6]:
le = LabelEncoder()
le = le.fit(train_df['action'])
train_df['action'] = le.transform(train_df['action'])
test_df['action'] = le.transform(test_df['action'])

In [7]:
img_path_list = []
for i in range(15):
    path_list = list(train_df[train_df['action']==i]['img_path'])
    if len(path_list) >= 5000:
        tmp = random.sample(path_list, 5000)
        for i in tmp:
            img_path_list.append(i)
    else:
        for i in path_list:
            img_path_list.append(i)
df = pd.DataFrame(img_path_list)
df.columns = ['img_path']
df

path_label_df = pd.merge(train_df, df, on='img_path', how='inner')
path_label_df

,action,img_path,user_x,lat,lon
0,14,./ETRI_data_RP_png/user03/1600707600/RP/160076...,user03,37.560580,126.980014
1,14,./ETRI_data_RP_png/user02/1600700400/RP/160076...,user02,37.521518,127.032711
2,11,./ETRI_data_RP_png/user03/1600192800/RP/160025...,user03,37.485350,126.977478
3,13,./ETRI_data_RP_png/user04/1600207800/RP/160025...,user04,37.475682,127.127883
4,3,./ETRI_data_RP_png/user04/1599690000/RP/159973...,user04,37.435011,127.138813
...,...,...,...,...,...
43508,14,./ETRI_data_RP_png/user04/1599085500/RP/159909...,user04,37.477714,127.122898
43509,8,./ETRI_data_RP_png/user01/1600095660/RP/160010...,user01,37.278513,127.163544
43510,7,./ETRI_data_RP_png/user03/1599510360/RP/159957...,user03,37.485431,126.977606
43511,5,./ETRI_data_RP_png/user04/1600380900/RP/160042...,user04,37.481213,127.142602


In [8]:
path_label_df['action'].value_counts()

14    5000
13    5000
8     5000
4     4381
6     3968
12    2534
3     2481
10    2406
7     2291
1     2187
11    2144
9     1995
5     1927
2     1580
0      619
Name: action, dtype: int64

In [9]:
img_path_list = []
for i in range(15):
    path_list = list(test_df[test_df['action']==i]['img_path'])
    if len(path_list) >= 300:
        tmp = random.sample(path_list, 300)
        for i in tmp:
            img_path_list.append(i)
    else:
        for i in path_list:
            img_path_list.append(i)
df2 = pd.DataFrame(img_path_list)
df2.columns = ['img_path']
df2

path_label_df2 = pd.merge(test_df, df2, on='img_path', how='inner')
path_label_df2

,action,img_path,user_x,lat,lon
0,13,./ETRI_data_RP_png/user03/1600876800/RP/160090...,user03,37.488120,126.982263
1,10,./ETRI_data_RP_png/user26/1599836400/RP/159985...,user26,37.554346,126.922069
2,6,./ETRI_data_RP_png/user27/1600309320/RP/160031...,user27,37.529723,127.008367
3,5,./ETRI_data_RP_png/user04/1599957060/RP/159997...,user04,37.434939,127.138877
4,11,./ETRI_data_RP_png/user02/1598972400/RP/159903...,user02,37.521560,127.032720
...,...,...,...,...,...
3724,13,./ETRI_data_RP_png/user03/1599928800/RP/159999...,user03,37.283530,126.980865
3725,8,./ETRI_data_RP_png/user03/1601141400/RP/160121...,user03,37.485415,126.977617
3726,13,./ETRI_data_RP_png/user03/1600448400/RP/160048...,user03,37.517630,127.089633
3727,4,./ETRI_data_RP_png/user04/1599690000/RP/159970...,user04,37.464091,127.127168


In [10]:
path_label_df2['action'].value_counts()

13    300
6     300
8     300
14    300
4     300
12    273
10    271
3     271
11    250
7     248
1     247
9     220
5     208
2     158
0      83
Name: action, dtype: int64

In [11]:
train_df = path_label_df
train_df

,action,img_path,user_x,lat,lon
0,14,./ETRI_data_RP_png/user03/1600707600/RP/160076...,user03,37.560580,126.980014
1,14,./ETRI_data_RP_png/user02/1600700400/RP/160076...,user02,37.521518,127.032711
2,11,./ETRI_data_RP_png/user03/1600192800/RP/160025...,user03,37.485350,126.977478
3,13,./ETRI_data_RP_png/user04/1600207800/RP/160025...,user04,37.475682,127.127883
4,3,./ETRI_data_RP_png/user04/1599690000/RP/159973...,user04,37.435011,127.138813
...,...,...,...,...,...
43508,14,./ETRI_data_RP_png/user04/1599085500/RP/159909...,user04,37.477714,127.122898
43509,8,./ETRI_data_RP_png/user01/1600095660/RP/160010...,user01,37.278513,127.163544
43510,7,./ETRI_data_RP_png/user03/1599510360/RP/159957...,user03,37.485431,126.977606
43511,5,./ETRI_data_RP_png/user04/1600380900/RP/160042...,user04,37.481213,127.142602


In [12]:
test_df = path_label_df
test_df

,action,img_path,user_x,lat,lon
0,14,./ETRI_data_RP_png/user03/1600707600/RP/160076...,user03,37.560580,126.980014
1,14,./ETRI_data_RP_png/user02/1600700400/RP/160076...,user02,37.521518,127.032711
2,11,./ETRI_data_RP_png/user03/1600192800/RP/160025...,user03,37.485350,126.977478
3,13,./ETRI_data_RP_png/user04/1600207800/RP/160025...,user04,37.475682,127.127883
4,3,./ETRI_data_RP_png/user04/1599690000/RP/159973...,user04,37.435011,127.138813
...,...,...,...,...,...
43508,14,./ETRI_data_RP_png/user04/1599085500/RP/159909...,user04,37.477714,127.122898
43509,8,./ETRI_data_RP_png/user01/1600095660/RP/160010...,user01,37.278513,127.163544
43510,7,./ETRI_data_RP_png/user03/1599510360/RP/159957...,user03,37.485431,126.977606
43511,5,./ETRI_data_RP_png/user04/1600380900/RP/160042...,user04,37.481213,127.142602


In [13]:
train, val, _, _ = train_test_split(train_df, train_df['action'], test_size=0.1, random_state=CFG['SEED'], stratify=train_df['action'])

In [14]:
train['action'].value_counts()

14    4500
13    4500
8     4500
4     3943
6     3571
12    2281
3     2233
10    2165
7     2062
1     1968
11    1930
9     1795
5     1734
2     1422
0      557
Name: action, dtype: int64

In [15]:
val['action'].value_counts()

14    500
13    500
8     500
4     438
6     397
12    253
3     248
10    241
7     229
1     219
11    214
9     200
5     193
2     158
0      62
Name: action, dtype: int64

In [16]:
# train, val, _, _ = train_test_split(train_df, train_df['action'], test_size=0.1, random_state=CFG['SEED'])

In [17]:
train

,action,img_path,user_x,lat,lon
41386,3,./ETRI_data_RP_png/user03/1600876800/RP/160095...,user03,37.485394,126.977561
36791,10,./ETRI_data_RP_png/user01/1601166300/RP/160119...,user01,37.482449,126.956348
18443,6,./ETRI_data_RP_png/user26/1600095600/RP/160015...,user26,37.278514,127.163543
27588,12,./ETRI_data_RP_png/user09/1600403400/RP/160041...,user09,37.381507,127.230470
7780,14,./ETRI_data_RP_png/user04/1600207800/RP/160024...,user04,37.477683,127.122614
...,...,...,...,...,...
10322,6,./ETRI_data_RP_png/user06/1601912160/RP/160198...,user06,37.513558,127.045732
10575,1,./ETRI_data_RP_png/user26/1600441200/RP/160049...,user26,37.242523,127.390785
21870,7,./ETRI_data_RP_png/user04/1600558200/RP/160058...,user04,37.434926,127.138867
23790,8,./ETRI_data_RP_png/user03/1599928800/RP/160000...,user03,37.485407,126.977599


In [18]:
val

,action,img_path,user_x,lat,lon
30089,10,./ETRI_data_RP_png/user28/1601159400/RP/160117...,user28,37.446267,126.680676
18084,14,./ETRI_data_RP_png/user04/1599085500/RP/159909...,user04,37.477711,127.122973
2810,3,./ETRI_data_RP_png/user03/1600192800/RP/160024...,user03,37.485415,126.977583
122,14,./ETRI_data_RP_png/user01/1600315920/RP/160034...,user01,37.483198,127.012350
29539,13,./ETRI_data_RP_png/user01/1600315920/RP/160032...,user01,37.484763,127.012016
...,...,...,...,...,...
19855,13,./ETRI_data_RP_png/user04/1599430860/RP/159943...,user04,37.469998,127.126735
29858,7,./ETRI_data_RP_png/user04/1600558200/RP/160058...,user04,37.434946,127.138855
28434,14,./ETRI_data_RP_png/user04/1598828400/RP/159885...,user04,37.477664,127.122784
38177,8,./ETRI_data_RP_png/user04/1599957060/RP/159998...,user04,37.434975,127.138894


In [19]:
RP_tfms = A.Compose([
    A.Resize(width=CFG['IMG_SIZE'], height=CFG['IMG_SIZE']),
    A.Normalize()
], p=1)

In [20]:
Gps_tfms = A.Compose([
    A.Resize(width=112, height=112),
    A.Normalize()
], p=1)

In [21]:
class RPDataset(Dataset):
    def __init__(self, df, rp_path_list, label_list, tfms=None):
        super().__init__()
        self.df = df
        self.rp_path_list = rp_path_list
        self.label_list = label_list
        self.tfms = tfms
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img = cv2.imread(self.rp_path_list[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image = self.tfms(image=img)['image']
        image = torch.tensor(np.array(image)).permute(2, 0, 1)
        
        if self.label_list is not None:
            label = self.label_list[idx]
            return image, label
        else:
            return image

In [22]:
class GpsDataset(Dataset):
    def __init__(self, df, lat_path_list, lon_path_list, label_list, tfms=None):
        super(GpsDataset, self).__init__()
        self.df = df
        self.lat_path_list = lat_path_list
        self.lon_path_list = lon_path_list
        self.label_list = label_list
        self.tfms = tfms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        lat = self.lat_path_list[idx]
        lon = self.lon_path_list[idx]
        feature_map = torch.tensor(np.array([lat, lon]))
        
        if self.label_list is not None:
            label = self.label_list[idx]
            return feature_map, label
        else:
            return feature_map

In [23]:
RP_train_dataset = RPDataset(df=train, rp_path_list=train['img_path'].values, label_list=train['action'].values, tfms=RP_tfms)
RP_train_loader = DataLoader(RP_train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

RP_val_dataset = RPDataset(df=val,rp_path_list=val['img_path'].values, label_list=val['action'].values, tfms=RP_tfms)
RP_val_loader = DataLoader(RP_val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [24]:
Gps_train_dataset = GpsDataset(df=train, lat_path_list=train['lat'].values, lon_path_list=train['lon'].values, label_list=train['action'].values, tfms=Gps_tfms)
Gps_train_loader = DataLoader(Gps_train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

Gps_val_dataset = GpsDataset(df=val, lat_path_list=train['lat'].values, lon_path_list=train['lon'].values, label_list=val['action'].values, tfms=Gps_tfms)
Gps_val_loader = DataLoader(Gps_val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [25]:
class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, weight=self.weight, reduction=self.reduction)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1-pt)**self.gamma * ce_loss).mean()
        return focal_loss

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        bias=False,
        padding = 1,
        padding_mode='zeros'
    )


def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=1,
        stride=stride,
        bias=False,
        padding = 1,
        padding_mode='zeros'
    )

class ChannelAttention(nn.Module):
    def __init__(self, channel):
        super(ChannelAttention, self).__init__()
        self.attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channel, channel, 1, padding=0, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(channel, channel, 1, padding=0, bias=False),
            nn.Sigmoid())
    def forward(self, x):
        y = self.attention(x)
        return x * y
    
class CAB(nn.Module):
    def __init__(self, channel):
        super(CAB, self).__init__()
        self.cab = nn.Sequential(
            nn.Conv2d(channel, channel, kernel_size=3, padding=1, stride=1),
            nn.GELU(),
            nn.Conv2d(channel, channel, kernel_size=3, padding=1, stride=1),
            ChannelAttention(channel)
            )
    def forward(self, x):
        return self.cab(x)


class IdentityBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride = 1):
        super(IdentityBlock, self).__init__()

        self.conv1 = conv3x3(in_planes, out_planes, stride)
#         self.cab = CAB(out_planes)
        self.conv2 = conv3x3(out_planes, out_planes, 1)

        self.bn1   = nn.BatchNorm2d(out_planes)
        self.bn2   = nn.BatchNorm2d(out_planes)

        self.shortcut = nn.Sequential()
        if stride != 1:
            self.shortcut = nn.Sequential(
                conv1x1(in_planes, out_planes, stride),
                nn.BatchNorm2d(out_planes)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out  = self.conv2(out)
#         out = self.cab(out)
        out  = self.bn2(out)
        out += identity
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, in_planes, num_blocks, num_classes):
        super(ResNet, self).__init__()

        self.in_planes = in_planes

#         self.conv = nn.Conv2d(3, in_planes, kernel_size = 3, stride = 1, padding = 1, padding_mode='zeros', bias=False)
        self.conv = nn.Conv2d(3, 32, kernel_size = 3, stride = 1, padding = 3, padding_mode='zeros', bias=False)
        self.cab = CAB(in_planes)
#         self.bn = nn.BatchNorm2d(self.in_planes)
        self.bn = nn.BatchNorm2d(32)
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer1 = self.make_layer(block, in_planes, num_blocks[0], stride=1)
        self.layer2 = self.make_layer(block, in_planes, num_blocks[1], stride=1)
        self.layer3 = self.make_layer(block, in_planes, num_blocks[2], stride=1)
        self.layer4 = self.make_layer(block, in_planes, num_blocks[3], stride=1)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.linear  = nn.Linear(in_planes + 2, num_classes)

    def make_layer(self, block, out_planes, num_blocks, stride):
            strides = [stride] + [1] * (num_blocks -1)
            layers = []
            for stride in strides:
                layers.append(block(self.in_planes, out_planes))
                self.in_planes = out_planes
            return nn.Sequential(*layers)

    def forward(self, x, g):
        out = self.conv(x)
#         out = self.cab(out)
        out = self.bn(out)
        out = F.relu(out)
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
#         print(out.shape)
        g = g.unsqueeze(-1).unsqueeze(-1)
        g = g.expand(-1, -1, 1, 1).float()
#         g = self.avgpool(g)
#         print(g.shape)
        out = torch.cat((out, g), dim=1)
        out = torch.flatten(out, 1)
        out = self.linear(out)
        return out


def ResNet18(in_planes, num_classes):
    return ResNet(block = IdentityBlock, in_planes = in_planes, num_blocks = [2, 2, 2, 2], num_classes = num_classes)

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F

in_planes = 64
num_classes = 15


class Residual(nn.Module):
    def __init__(self, numIn, numOut, stride = 1):
        super(Residual, self).__init__()
        self.numIn = numIn
        self.numOut = numOut
        self.stride = stride
        self.conv1 = nn.Conv2d(self.numIn, self.numOut, bias = False, kernel_size = 3,stride = self.stride,padding = 1)
        self.bn1 = nn.BatchNorm2d(self.numOut)
        self.relu = nn.ReLU(inplace = True)
        self.conv2 = nn.Conv2d(self.numOut, self.numOut, bias = False, kernel_size = 3, stride = self.stride, padding = 1)
        self.bn2 = nn.BatchNorm2d(self.numOut)
        
        if self.numIn != self.numOut:
            self.conv4 = nn.Conv2d(self.numIn, self.numOut, bias = True, kernel_size = 1)
            
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        
        if self.numIn != self.numOut:
            residual = self.conv4(x)
        
        return out + residual
    
class CAB(nn.Module):
    def __init__(self, channel):
        super(CAB, self).__init__()
        self.cab = nn.Sequential(
            nn.Conv2d(channel, channel, kernel_size=3, padding=1, stride=1),
            nn.GELU(),
            nn.Conv2d(channel, channel, kernel_size=3, padding=1, stride=1),
            ChannelAttention(channel)
            )
    def forward(self, x):
        return self.cab(x)
    

class  ResNet_CAM(nn.Module):
    def __init__(self,nOut):
        super(ResNet_CAM, self).__init__()
        self.cab = CAB(32)
        self.nOut = nOut
        self.conv1 = nn.Conv2d(3, 32, kernel_size = 3, stride = 1, padding = 3,bias = False)#320
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace = True)
        self.maxpool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.res1 = Residual(32,32)
        self.res2 = Residual(32,32)
        
        self.res3 = Residual(32,32)
        self.res4 = Residual(32,32)
        
        self.res5 = Residual(32,32)
        self.res6 = Residual(32,32)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.linear  = nn.Linear(in_planes//2 + 2, num_classes)
        
        self.lr1 = nn.Linear(64*16*16,256)
        self.gap = nn.AvgPool2d(kernel_size = 14, stride = 1)
        self.lr1 = nn.Linear(32,nOut)
        self.lr2 = nn.Linear(256,nOut)

    def forward(self, out, g):#Bx3X224x224
        out = self.conv1(out)#Bx64x224x224
        out = self.cab(out)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)#Bx64x112x112
        out = self.res1(out)
        out = self.res2(out)
        out = self.maxpool(out)#Bx64x56x56
        out = self.res3(out)
        out = self.res4(out)
        out = self.maxpool(out)#Bx64x28x28
        out = self.res5(out)
        out = self.res6(out)
        out = self.maxpool(out)#Bx64x14x14
        out = self.avgpool(out)
        g = g.unsqueeze(-1).unsqueeze(-1)
        g = g.expand(-1, -1, 1, 1).float()
        out = torch.cat((out, g), dim=1)
        out = torch.flatten(out, 1)
        out = self.linear(out)
            
        return out

In [32]:
def train(model, optimizer, scheduler, device):
    model.to(device)
#     criterion = FocalLoss().to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for i, data in enumerate(zip(tqdm(RP_train_loader), Gps_train_loader)):
            data1, data2 = data
            images, labels = data1
            gps, _ = data2
            
            images = images.to(device)
            gps = gps.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(images, gps)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

In [33]:
def validation(model, criterion, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for i, data in enumerate(zip(tqdm(RP_val_loader), Gps_val_loader)):
            data1, data2 = data
            images, labels = data1
            gps, _ = data2
            
            images = images.to(device)
            gps = gps.to(device)
            labels = labels.to(device)
            
            logit = model(images, gps)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='micro')
    return _val_loss, _val_score

In [ ]:
# model = resnet.ResNet18(64, 15)
# model = ResNet18(64, 15)
model = ResNet_CAM(15)
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, scheduler, device)

100%|████████████████████████████████████████████████████████████████| 136/136 [00:25<00:00,  5.40it/s]


Epoch [1], Train Loss : [5.14810] Val Loss : [2.43387] Val F1 : [0.21117]


100%|████████████████████████████████████████████████████████████████| 136/136 [00:25<00:00,  5.31it/s]


Epoch [2], Train Loss : [2.40986] Val Loss : [2.36196] Val F1 : [0.24426]


100%|████████████████████████████████████████████████████████████████| 136/136 [00:25<00:00,  5.30it/s]


Epoch [3], Train Loss : [2.34329] Val Loss : [2.33462] Val F1 : [0.25437]


 44%|███████████████████████████▋                                   | 539/1224 [02:24<03:03,  3.73it/s]

In [ ]:
RP_test_dataset = RPDataset(df=test_df, rp_path_list=test_df['img_path'].values, label_list=test_df['action'].values, tfms=RP_tfms)
RP_test_loader = DataLoader(RP_test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

Gps_test_dataset = GpsDataset(df=test_df, lat_path_list=test_df['lat'].values, lon_path_list=test_df['lon'].values, label_list=test_df['action'].values, tfms=Gps_tfms)
Gps_test_loader = DataLoader(Gps_test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

In [ ]:
def inference(model, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for i, data in enumerate(zip(tqdm(RP_test_loader), Gps_test_loader)):
            data1, data2 = data
            images, labels = data1
            gps, _ = data2
            
            images = images.to(device)
            gps = gps.to(device)
            labels = labels.to(device)
            
            logit = model(images, gps)
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [ ]:
preds = inference(model, device)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(test_df['action'], preds, labels=[x for x in range(0, 15)])
confusion_matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize = (25,25))
plt.title('Confusion Matrix')

sns.heatmap(confusion_matrix, annot=True)

In [ ]:
from sklearn.metrics import f1_score 
f1 = f1_score(test_df['action'], preds, average='micro')
print('F1-score: {0:.4f}'.format(f1))

In [ ]:
torch.save(model, f'./save_model/0424_RPmGps_lr1e3_cab_2.pt')

In [ ]:
from sklearn.metrics import classification_report
y_true = test_df['action']
y_pred = preds
target_names = [str(x) for x in range(15)]
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
y_true = test_df['action']
y_pred = preds
target_names = [str(x) for x in range(15)]
print(classification_report(y_true, y_pred, target_names=target_names))